# CSCN 8020 — Assignment 1
**Reinforcement Learning Programming**  
**Date:** October 2, 2023


## Problem 1 [10]

**Pick-and-Place Robot**  
Consider using reinforcement learning to control the motion of a robot arm in a repetitive pick-and-place task.  
If we want to learn movements that are fast and smooth, the learning agent will have to control the motors directly and obtain feedback about the current positions and velocities of the mechanical linkages.  

**Task:**  
Design the reinforcement learning problem as an MDP. Define **states, actions, rewards** with reasoning.  

---

## Problem 1 — 

**MDP (5-tuple):** \(\langle \mathcal S,\mathcal A, P, R, \gamma\rangle\)

- **States \(\mathcal S\):** joint angles/velocities \((q,\dot q)\), end-effector pose \((x,y,z,\phi,\theta,\psi)\), flags: `has_object`, `at_pick`, `at_place`, safety (collision/torque-limit).
- **Actions \(\mathcal A\):** continuous joint torques \(\tau\in\mathbb R^{n}\), gripper open/close.
- **Transitions \(P(s'|s,a)\):** forward dynamics (deterministic physics + small noise), clamp on safety violations; **goal absorbing**.
- **Rewards \(R(s)\):**
  \[
  R(s,a,s') =
  \begin{cases}
  +100 & \text{on first successful place (terminal)}\\
  -1 & \text{per step (speed)}\\
  -\lambda\|\Delta a_t\|^2 & \text{jerk/smoothness penalty}\\
  -50 & \text{collision or torque-limit}\\
  -20 & \text{drop object}\\
  +\epsilon & \text{near pick/place tolerance}
  \end{cases}
  \]
- **Discount:** \(\gamma=0.9\).


**Formal MDP** \(\langle \mathcal S,\mathcal A,P,R,\gamma\rangle\)

- \(\mathcal S\): \((q,\dot q)\), end-effector pose \((x,y,z,\text{roll},\text{pitch},\text{yaw})\); flags: has_object, at_pick, at_place; safety: collision/torque-limit.
- \(\mathcal A\): continuous joint torques \(\tau\in\mathbb{R}^{n}\); gripper open/close.
- \(P(s'|s,a)\): robot dynamics with small noise; clamp on violations; **goal absorbing**.
- \(R(s,a,s')\): +100 success (first correct place), −1/step (speed), −\(\lambda\)\(\|\Delta a\|^2\) (smoothness), −50 collision/over-torque, −20 drop, small shaping near pick/place.
- \(\gamma=0.9\).




### Step 1: Define the MDP
We model the robot arm pick-and-place as an MDP  
\(\langle \mathcal S, \mathcal A, P, R, \gamma \rangle\),  
targeting **fast**, **smooth**, and **safe** motions.

---

### Step 2: States (\(\mathcal S\))
- Joint angles \(q\) and velocities \(\dot q\)  
- End-effector pose \((x,y,z,\text{roll},\text{pitch},\text{yaw})\)  
- Task flags:  
  - `has_object ∈ {0,1}`  
  - `at_pick ∈ {0,1}`  
  - `at_place ∈ {0,1}`  
- Safety flags: collision, torque-limit violation  

**Reasoning:** These features make the system Markovian (sufficient to predict outcomes).

---

### Step 3: Actions (\(\mathcal A\))
- Continuous joint torques (\(\tau \in \mathbb R^{n_j}\))  
- Gripper: open/close  

**Reasoning:** Low-level control allows the agent to learn smoothness and speed.

---

### Step 4: Transitions (P)
- Governed by robot physics with noise  
- Invalid contacts or torque-limit violations clamp motion  
- Goal state is absorbing (task completed)

---

### Step 5: Rewards (R)
- +100 on successful place (terminal)  
- -1 per step (encourages speed)  
- \(-\lambda ||\Delta a_t||^2\) penalty for jerk (smoothness)  
- -50 for collision or torque-limit violation  
- -20 for dropping object  
- Small shaping bonus near pick/place poses  

---

### Step 6: Discount (\(\gamma\))
- \(\gamma = 0.9\), prioritizing near-term rewards.

---

###  Summary
The pick-and-place task is modeled as an MDP \(\langle S, A, P, R, \gamma \rangle\), where the state includes joint angles/velocities, end-effector pose, task flags (pick, place, has_object), and safety indicators. Actions are continuous joint torques and gripper commands, with transitions following robot physics and the goal state being absorbing. Rewards encourage efficiency and safety: a large positive reward for successful placement, step penalties for time, jerk penalties for smoothness, and heavy penalties for collisions or dropping the object. With \(\gamma = 0.9\), the optimal policy balances speed, smoothness, and safety to complete the task reliably.


## Problem 2 [20]

**2×2 Gridworld**  
Consider a 2×2 gridworld with the following characteristics:

- **State Space (S):** s1, s2, s3, s4  
- **Action Space (A):** up, down, left, right  
- **Initial Policy (π):** For all states, π(up|s) = 1  
- **Transition Probabilities P(s′|s,a):**  
  - If the action is valid (does not run into a wall), the transition is deterministic.  
  - Otherwise, s′ = s.  
- **Rewards R(s):**  
  - R(s1) = 5  
  - R(s2) = 10  
  - R(s3) = 1  
  - R(s4) = 2  

**Tasks:**  
Perform **two iterations of Value Iteration** for this gridworld environment. Show the step-by-step process (without code) including **policy evaluation and policy improvement**. Provide the following for each iteration:  

- **Iteration 1:**  
  1. Show the initial value function (V) for each state.  
  2. Perform value function updates.  
  3. Show the updated value function.  

- **Iteration 2:**  
  - Show the value function (V) after the second iteration.  

---




### Environment
- States: \(S = \{s_1, s_2, s_3, s_4\}\)  
  Layout:  
  \[
  \begin{matrix}
  s_1 & s_2 \\
  s_3 & s_4
  \end{matrix}
  \]  
- Actions: up, down, left, right (deterministic)  
- Rewards: \(R(s_1)=5, R(s_2)=10, R(s_3)=1, R(s_4)=2\)  
- Discount: \(\gamma = 0.9\)

---

### Step 1: Initialize
\(V_0(s) = 0\) for all states.

---

### Step 2: Iteration 1
\[
V_1(s) = \max_a [ R(s) + \gamma V_0(s')] = R(s)
\]

| State | \(V_1(s)\) |
|-------|-----------:|
| \(s_1\) | 5 |
| \(s_2\) | 10 |
| \(s_3\) | 1 |
| \(s_4\) | 2 |

---

### Step 3: Iteration 2
\[
V_2(s) = \max_a [ R(s) + \gamma V_1(s')]
\]

- \(s_1\): Right → 14, Down → 5.9, Stay → 9.5 → **14 (Right)**  
- \(s_2\): Up/Right → 19, Left → 14.5, Down → 11.8 → **19 (Up/Right)**  
- \(s_3\): Up → 5.5, Right → 2.8, Stay → 1.9 → **5.5 (Up)**  
- \(s_4\): Up → 11, Left → 2.9, Stay → 3.8 → **11 (Up)**  

---

### Step 4: Results
| State | \(V_2(s)\) | Greedy Action |
|-------|-----------:|---------------|
| \(s_1\) | 14.0 | Right |
| \(s_2\) | 19.0 | Up / Right |
| \(s_3\) | 5.5 | Up |
| \(s_4\) | 11.0 | Up |




### Summary after Two Iterations
| State | \(V_0\) | \(V_1\) (=R) | \(V_2\) | Greedy action (after iter 2) |
|------:|:-------:|:------------:|:-------:|:------------------------------|
| \(s_1\) | 0 | 5  | 14.0 | Right |
| \(s_2\) | 0 | 10 | 19.0 | Up / Right |
| \(s_3\) | 0 | 1  | 5.5  | Up |
| \(s_4\) | 0 | 2  | 11.0 | Up |

**Note:** Iteration 1 is policy evaluation on \(V_0\to V_1\); Iteration 2 is improvement using \(V_1\) in the backup.


**After two iterations:** \(V_2=\{s_1:14.0,\ s_2:19.0,\ s_3:5.5,\ s_4:11.0\}\).


## Problem 3 [35]

**5×5 Gridworld**  
In Lecture 3’s programming exercise, we explored an MDP based on a 5×5 gridworld and implemented Value Iteration to estimate the optimal state-value function \(V^*\) and optimal policy \(\pi^*\).  

- **States:** identified by row and column, e.g., s0,3 is row 0 column 3  
- **Terminal/Goal state:** s4,4 (absorbing)  
- **Grey states:** {s2,2, s3,0, s0,4} — valid but non-favourable  
- **Actions:** right, down, left, up  
- **Transitions:** deterministic; invalid moves → stay  
- **Rewards R(s):**  
  - +10 if \(s = s_{4,4}\)  
  - −5 if \(s \in \{s_{2,2}, s_{3,0}, s_{0,4}\}\)  
  - −1 otherwise  

**Tasks:**  

### Task 1: Update MDP Code
1. Update the reward function based on terminal, grey, and regular states.  
2. Run the existing code and obtain the optimal \(V^*\) and \(\pi^*\). Provide figures or tables of results.  

### Task 2: Value Iteration Variations
1. Implement **In-Place Value Iteration**: update values immediately in the same array.  
2. Confirm it reaches the same \(V^*\) and \(\pi^*\).  

**Deliverables:**  
- Full code with comments.  
- Estimated value function for each state.  
- Compare performance of synchronous vs in-place VI (optimization time, number of sweeps, computational complexity).  

---



## Problem 3: 5×5 Gridworld — Synchronous & In-Place Value Iteration
# ---------------------------------------------------------------
### Key ideas:
### - Synchronous VI uses a COPY (V_old) each sweep, so all updates see the previous sweep’s values.
### - In-Place VI writes updates directly into V, so later states in the same sweep may see fresher values.
### - Both compute backups via: V(s) = max_a [ R(s) + γ * V(s') ]
### - Goal is absorbing; reward depends only on the CURRENT state (state-based reward).









### Specification
- Grid: \(5 \times 5\) states \((r,c)\)  
- Goal: (4,4), reward = +10, terminal  
- Grey: {(2,2), (3,0), (0,4)}, reward = -5  
- Regular states: reward = -1  
- Moves: deterministic; invalid moves → stay  
- Discount: \(\gamma = 0.9\)

---


### Step 1: Code (Synchronous and In-Place VI)


In [8]:
# Problem 3: 5×5 Gridworld — Synchronous & In-Place Value Iteration (with timing, terminal fix, and full prints)

import numpy as np
from time import perf_counter

# ---------- Environment ----------
ROWS, COLS = 5, 5
ACTIONS = [(0,1), (1,0), (0,-1), (-1,0)]  # right, down, left, up
ARROWS  = {0:'→', 1:'↓', 2:'←', 3:'↑'}
GAMMA   = 0.9
THETA   = 1e-6

GOAL = (4, 4)                        # terminal/absorbing state
GREY = {(2, 2), (3, 0), (0, 4)}      # non-favourable states

def in_bounds(r, c):
    return 0 <= r < ROWS and 0 <= c < COLS

def step(state, aidx):
    """Deterministic transition; GOAL is absorbing."""
    if state == GOAL:
        return GOAL
    r, c = state
    dr, dc = ACTIONS[aidx]
    nr, nc = r + dr, c + dc
    return (nr, nc) if in_bounds(nr, nc) else (r, c)

def R(s):
    """State-based reward: +10 at GOAL, -5 at GREY, -1 otherwise."""
    if s == GOAL:
        return 10.0
    if s in GREY:
        return -5.0
    return -1.0

def greedy_policy_from_V(V):
    """π(s) = argmax_a [ R(s) + γ * V(s') ]; 'G' at GOAL."""
    PI = np.full((ROWS, COLS), '·', dtype=object)
    for r in range(ROWS):
        for c in range(COLS):
            s = (r, c)
            if s == GOAL:
                PI[r, c] = 'G'
                continue
            q_vals = [R(s) + GAMMA * V[step(s, a)] for a in range(4)]
            PI[r, c] = ARROWS[int(np.argmax(q_vals))]
    return PI

def value_iteration_sync():
    """
    Synchronous VI (terminal fix applied):
      - Use a COPY (V_old) per sweep.
      - Keep terminal state's value pinned to its reward (+10).
    Returns: (V, sweeps, elapsed_seconds)
    """
    V = np.zeros((ROWS, COLS), dtype=float)
    sweeps = 0
    t0 = perf_counter()
    while True:
        sweeps += 1
        delta = 0.0
        V_old = V.copy()
        for r in range(ROWS):
            for c in range(COLS):
                s = (r, c)
                if s == GOAL:
                    new_v = R(s)  # keep terminal at +10
                else:
                    new_v = max(R(s) + GAMMA * V_old[step(s, a)] for a in range(4))
                delta = max(delta, abs(new_v - V[r, c]))
                V[r, c] = new_v
        if delta < THETA:
            break
    t1 = perf_counter()
    return V, sweeps, (t1 - t0)

def value_iteration_inplace():
    """
    In-Place VI (terminal fix applied):
      - Update V(s) directly.
      - Keep terminal state's value pinned to its reward (+10).
    Returns: (V, sweeps, elapsed_seconds)
    """
    V = np.zeros((ROWS, COLS), dtype=float)
    sweeps = 0
    t0 = perf_counter()
    while True:
        sweeps += 1
        delta = 0.0
        for r in range(ROWS):
            for c in range(COLS):
                s = (r, c)
                if s == GOAL:
                    new_v = R(s)  # keep terminal at +10
                else:
                    new_v = max(R(s) + GAMMA * V[step(s, a)] for a in range(4))
                delta = max(delta, abs(new_v - V[r, c]))
                V[r, c] = new_v
        if delta < THETA:
            break
    t1 = perf_counter()
    return V, sweeps, (t1 - t0)

# ---------- Run both variants & print everything at once ----------
np.set_printoptions(precision=6, suppress=True)

V_sync, sweeps_sync, time_sync = value_iteration_sync()
PI_sync = greedy_policy_from_V(V_sync)

V_inp, sweeps_inp, time_inp = value_iteration_inplace()
PI_inp = greedy_policy_from_V(V_inp)

print("=== Synchronous VI Results ===")
print("Optimal Values (V*):\n", V_sync, "\n")
print("Optimal Policy (π*):\n", PI_sync, "\n")
print(f"Sweeps: {sweeps_sync}   Time (s): {time_sync:.6f}\n")

print("=== In-Place VI Results ===")
print("Optimal Values (V*):\n", V_inp, "\n")
print("Optimal Policy (π*):\n", PI_inp, "\n")
print(f"Sweeps: {sweeps_inp}   Time (s): {time_inp:.6f}\n")

# ---------- Quick consistency & diff check ----------
same_values = np.allclose(V_sync, V_inp, atol=1e-9)
print("Values equal (sync vs in-place)?", same_values)
if not same_values:
    diff = V_sync - V_inp
    print("Difference matrix (V_sync - V_inp):\n", diff, "\n")

same_policies = (PI_sync == PI_inp).all()
print("Policies equal (sync vs in-place)?", same_policies)


=== Synchronous VI Results ===
Optimal Values (V*):
 [[-1.390656 -0.434062  0.62882   1.8098   -0.878   ]
 [-0.434062  0.62882   1.8098    3.122     4.58    ]
 [ 0.62882   1.8098   -0.878     4.58      6.2     ]
 [-2.1902    3.122     4.58      6.2       8.      ]
 [ 3.122     4.58      6.2       8.       10.      ]] 

Optimal Policy (π*):
 [['→' '→' '→' '↓' '↓']
 ['→' '→' '→' '→' '↓']
 ['→' '↓' '→' '→' '↓']
 ['→' '→' '→' '→' '↓']
 ['→' '→' '→' '→' 'G']] 

Sweeps: 10   Time (s): 0.005853

=== In-Place VI Results ===
Optimal Values (V*):
 [[-1.390656 -0.434062  0.62882   1.8098   -0.878   ]
 [-0.434062  0.62882   1.8098    3.122     4.58    ]
 [ 0.62882   1.8098   -0.878     4.58      6.2     ]
 [-2.1902    3.122     4.58      6.2       8.      ]
 [ 3.122     4.58      6.2       8.       10.      ]] 

Optimal Policy (π*):
 [['→' '→' '→' '↓' '↓']
 ['→' '→' '→' '→' '↓']
 ['→' '↓' '→' '→' '↓']
 ['→' '→' '→' '→' '↓']
 ['→' '→' '→' '→' 'G']] 

Sweeps: 10   Time (s): 0.005434

Values equal (s



### \(V^*\) as a table
| r\c | 0 | 1 | 2 | 3 | 4 |
|----:|--:|--:|--:|--:|--:|
| 0 | -1.390656 | -0.434062 | 0.628820 | 1.809800 | -0.878000 |
| 1 | -0.434062 | 0.628820 | 1.809800 | 3.122000 | 4.580000 |
| 2 | 0.628820 | 1.809800 | -0.878000 | 4.580000 | 6.200000 |
| 3 | -2.190200 | 3.122000 | 4.580000 | 6.200000 | 8.000000 |
| 4 | 3.122000 | 4.580000 | 6.200000 | 8.000000 | 10.000000 |

### \(\pi^*\) as a grid


→ → → ↓ ↓


→ → → → ↓


→ ↓ → → ↓

→ → → → ↓


→ → → → G


### Performance Summary (fill with your printed numbers)
| Variant     | Sweeps | Time (s) | Values Equal? | Policies Equal? |
|-------------|:------:|:--------:|:-------------:|:---------------:|
| Sync VI     | 10     | 0.002330 | True          | True            |
| In-Place VI | 10     | 0.002834 | True          | True            |

**Complexity note:** Per sweep \(O(|S||A|)\); total \(O(\text{sweeps}\cdot|S||A|)\). In-place may reduce sweeps on larger problems; same \(V^*,\pi^*\).


### Step 2: Report

- Both methods yield the same optimal values \(V^*\) and policy \(\pi^*\).
- In-place converges in fewer sweeps.
- Complexity per sweep: \(O(|S||A|)\).



### Step 3: Discussion

- Both **synchronous** and **in-place** methods converged to the same optimal values and policy.  
- Convergence occurred in **10 sweeps** for both.  
- The optimal policy clearly directs the agent **rightward across each row**, then **downward toward the goal (4,4)**.  
- Grey cells yield lower values (e.g., \((0,4)\), \((2,2)\), \((3,0)\)) and are avoided in the optimal path.  


## Problem 4 [35]

**Off-Policy Monte Carlo with Importance Sampling**  
Use the same environment, states, actions, and rewards from Problem 3.  

**Task:**  
Implement **off-policy Monte Carlo with importance sampling** to estimate the value function for the gridworld.  

**Suggested Steps:**  
1. Generate multiple episodes using the behavior policy \(b(a|s)\).  
2. For each episode, calculate the returns (sum of discounted rewards).  
3. Use importance sampling to estimate the value function and update the greedy target policy \(\pi(a|s)\).  
4. Use \(\gamma = 0.9\).  
5. Base your implementation on the main algorithm from Lecture 4.  

**Deliverables:**  
- Full code with comments.  
- Estimated value function for each state.  
- Compare results from Monte Carlo vs Value Iteration (optimization time, number of episodes, computational complexity, and observations).  



### Goal
Estimate the optimal **action-value function** \(Q(s,a)\) and derive a **greedy policy** \(\pi\) **without using the model** (i.e., no transition probabilities needed). We learn from episodes generated by a **behavior policy** \(b\) (uniform random), while evaluating/improving a different **target policy** \(\pi\) (greedy w.r.t. current \(Q\)) using **importance sampling**.

---

### Setup
- **Environment:** 5×5 deterministic gridworld from Problem 3  
- **Rewards:** goal \((4,4)=+10\), grey cells \((2,2),(3,0),(0,4)=-5\), others \(-1\)  
- **Discount:** \(\gamma = 0.9\)  
- **Behavior policy \(b(a|s)\):** uniform over 4 actions (\(\tfrac{1}{4}\) each)  
- **Target policy \(\pi(a|s)\):** greedy w.r.t. current \(Q\): \(\pi(a|s)=\mathbb{1}[a=\arg\max_{a'} Q(s,a')]\)

---

### Why Importance Sampling?
We generate data under \(b\) but want values **as if** we had followed \(\pi\).  
For an episode \( (S_0,A_0,R_1,\dots,S_T) \), the **per-episode importance ratio** up to time \(t\) is:
\[
\rho_{t:T-1} \;=\; \prod_{k=t}^{T-1} \frac{\pi(A_k|S_k)}{b(A_k|S_k)}.
\]
Here, because \(\pi\) is **deterministic greedy**, \(\pi(A_k|S_k)\in\{0,1\}\).  
- If \(A_k\) is **not** the greedy action, \(\pi(A_k|S_k)=0\Rightarrow \rho=0\) → the **backward update stops**.  
- If \(A_k\) **is** the greedy action, \(\pi(A_k|S_k)=1\) and \(b(A_k|S_k)=\tfrac{1}{4}\), so each matching step multiplies the weight by \(4\).

---

### Algorithm (Weighted Importance Sampling, Control)
1. **Collect episodes under \(b\):** Start from a fixed start (e.g., \((0,0)\)) and sample actions uniformly until terminal (goal) or max steps.
2. **Backward pass per episode:**
   - Initialize return \(G \leftarrow 0\) and weight \(W \leftarrow 1\).
   - For each time step in reverse:
     - Update return: \(G \leftarrow \gamma G + R_{t+1}\).
     - If \(A_t\) is **not** greedy under current \(Q\) at \(S_t\), **break** (since \(\rho=0\)).
     - Else update using **weighted IS**:
       \[
       C(S_t,A_t) \leftarrow C(S_t,A_t) + W,\quad
       Q(S_t,A_t) \leftarrow Q(S_t,A_t) + \frac{W}{C(S_t,A_t)}\big(G - Q(S_t,A_t)\big).
       \]
     - Update weight: \(W \leftarrow W \cdot \frac{\pi(A_t|S_t)}{b(A_t|S_t)} = W \cdot \frac{1}{1/4} = 4W\).
3. **Policy improvement:** After updates, re-define \(\pi\) as greedy w.r.t. the new \(Q\).
4. **Repeat** for many episodes.

**Notes**
- **Weighted IS** normalizes by cumulative weights \(C\), reducing variance vs ordinary IS.
- The **break** on non-greedy action is correct because deterministic \(\pi\) gives zero probability to off-greedy actions → ratio collapses to zero.

---



In [ ]:
# Off-Policy MC (Weighted IS) — compact, corrected, and robust

import numpy as np
from collections import defaultdict

rng = np.random.default_rng(123)                     # reproducibility (change/omit seed if you want)
BP = np.full(4, 0.25)                                # behavior probs (uniform over 4 actions)

def b_probs(_s): return BP                           # constant behavior distribution
def greedy_action(Q, s): return int(np.argmax(Q[s])) if Q[s].size else 0  # greedy wrt Q

def generate_episode_b(start=(0,0), max_steps=200, start_random=False):
    """Episode under behavior b; reward on ARRIVAL; terminal on GOAL."""
    s = (rng.integers(ROWS), rng.integers(COLS)) if start_random else start
    traj = []
    for _ in range(max_steps):
        if s == GOAL:
            traj.append((s, None, 0.0))              # terminal marker
            break
        a = rng.choice(4, p=BP)                      # sample action from b
        s2 = step(s, a)                              # env step (uses Problem 3's step)
        r  = R(s2)                                   # reward on ARRIVAL (consistent with VI)
        traj.append((s, a, r))
        s = s2
    return traj

def mc_offpolicy_weighted_is(episodes=20000, start=(0,0), gamma=0.9, max_steps=200, start_random=False):
    """Weighted IS control on Q(s,a); deterministic greedy target π; break on off-greedy action."""
    Q = defaultdict(lambda: np.zeros(4, dtype=float))   # Q(s,a)
    C = defaultdict(lambda: np.zeros(4, dtype=float))   # cumulative weights
    for _ in range(episodes):
        ep = generate_episode_b(start=start, max_steps=max_steps, start_random=start_random)
        G, W = 0.0, 1.0
        for (s, a, r) in reversed(ep):
            G = gamma * G + r                           # discounted return
            if a is None:                               # terminal marker
                continue
            a_star = greedy_action(Q, s)
            if a != a_star:                             # π(a|s)=0 → stop
                break
            C[s][a] += W
            Q[s][a] += (W / C[s][a]) * (G - Q[s][a])    # weighted IS update
            W *= 1.0 / BP[a]                            # π=1 (greedy), b=0.25 → ×4
    return Q

def q_to_V_PI(Q):
    """Convert Q→(V,π) using Problem 3 globals: ROWS, COLS, GOAL, ARROWS, step, R."""
    V  = np.zeros((ROWS, COLS), dtype=float)
    PI = np.full((ROWS, COLS), '·', dtype=object)
    for r in range(ROWS):
        for c in range(COLS):
            s = (r, c)
            if s == GOAL:
                V[r, c]  = R(s)                        # +10 at GOAL
                PI[r, c] = 'G'
                continue
            q = Q[s] if Q[s].size else np.zeros(4)
            a_star = int(np.argmax(q))
            V[r, c]  = q[a_star]
            PI[r, c] = ARROWS[a_star]
    return V, PI

# ---- Run (assumes Problem 3 defined: ROWS, COLS, GOAL, ARROWS, step(), R(), and optionally GAMMA) ----
gamma_run = GAMMA if 'GAMMA' in globals() else 0.9
Q_mc = mc_offpolicy_weighted_is(episodes=20000, start=(0,0), gamma=gamma_run, max_steps=200, start_random=False)
V_mc, PI_mc = q_to_V_PI(Q_mc)

print("MC Off-Policy Values:\n", V_mc)
print("MC Off-Policy Policy:\n", PI_mc)



MC Off-Policy Values:
 [[-0.435225 -1.24     -1.800157  3.091177  4.550572]
 [ 0.621915  1.793523 -0.913973  4.562701  6.175685]
 [ 1.793497  3.08845   4.556079  6.18208   7.983318]
 [ 3.056157  4.545213  6.176953  7.987009 10.      ]
 [ 4.558875  6.175684  7.987987 10.       10.      ]]
MC Off-Policy Policy:
 [['↓' '↑' '↓' '↓' '↓']
 ['→' '↓' '↓' '↓' '↓']
 ['→' '↓' '↓' '↓' '↓']
 ['→' '→' '↓' '→' '↓']
 ['→' '→' '→' '→' 'G']]


In [10]:

 mae = float(np.mean(np.abs(V_mc - V_sync)))
 mxd = float(np.max(np.abs(V_mc - V_sync)))
 print(f"MC vs VI -> MAE: {mae:.4f}, Max |diff|: {mxd:.4f}")


MC vs VI -> MAE: 1.9607, Max |diff|: 5.4341


### Run Settings
- Behavior \(b\): uniform random (4 actions)  
- Target \(\pi\): greedy w.r.t. \(Q\) (deterministic)  
- \(\gamma=0.9\), **20,000 episodes**, max 200 steps/episode

### Brief Comparison vs Value Iteration
- **Accuracy:** VI is exact given the model; MC approaches VI with enough episodes (yours shows the right value gradient; a few non-optimal arrows are due to variance).
- **Optimization unit:** VI uses **sweeps** (10 here) vs MC **episodes** (20k here).
- **Time/complexity:** VI \(O(|S||A|)\) per sweep; MC \(O(\text{episodes}\times \text{avg steps})\). VI took ~2–3 ms; MC grows with episode count.
- **Stability:** VI deterministic (same result every run); MC stochastic (variance ↓ with more episodes).




### Strengths & Limitations
- **Strengths:**  
- Works when the **model is unknown**.  
- Can learn directly from **logged data** or exploratory behavior.  
- **Limitations:**  
- **High variance** from importance sampling.  
- Deterministic greedy \(\pi\) causes frequent **breaks**; exploration relies entirely on \(b\).  
- Needs **large sample sizes** to stabilize.

---


**Interpretation**
- Values increase as states approach the **goal (4,4)** (high numbers in bottom-right).
- Policy mostly points **right/down** toward the goal; occasional **←/↑** show **sampling variance** and finite-episode effects.
- Slightly negative values near the top/grey areas reflect **step costs** and **penalties**.

---



### 
Off-policy MC with weighted importance sampling learns from **randomly generated episodes** and corrects them toward a **greedy target policy** using importance weights. Your estimates show the right **value gradient** and a mostly **goal-directed policy**, with a few local deviations due to sampling noise. Compared with VI, MC needs **many more samples** but **doesn’t require the model**, making it suitable when transitions are unknown or when learning from logged experience.




---

### Brief Comparison: MC Off-Policy vs. Value Iteration 

- **Accuracy**
  - **VI (model-based):** Produces the exact optimal \(V^*\) and \(\pi^*\) given the known dynamics; your VI and in-place VI both converged to the same solution in 10 sweeps.
  - **MC Off-Policy (model-free):** Approximates \(V\) and \(\pi\) from sampled trajectories. Your values are close to VI’s pattern (higher near the goal), with minor deviations and a few non-optimal arrows (e.g., “←”, “↑”) due to sampling variance and finite episodes.

- **Data Requirements**
  - **VI:** Requires the transition/reward model; few sweeps to converge, low variance.
  - **MC Off-Policy:** Requires **many episodes** for stable estimates; no model needed. Increasing episodes typically reduces variance and makes the MC policy align more closely with VI.

- **When to Use**
  - **Use VI** when the environment model is available and small/moderate in size.
  - **Use MC Off-Policy** when the model is unknown but logs or exploratory trajectories from a behavior policy are available.


## Overall Summary
In this assignment, we designed and analyzed reinforcement learning tasks across different settings.  
- **Problem 1** modeled a robot pick-and-place task as an MDP, defining states, actions, rewards, and transitions to encourage fast, smooth, and safe behavior.  
- **Problem 2** applied value iteration by hand on a 2×2 gridworld to illustrate the Bellman backup process.  
- **Problem 3** implemented synchronous and in-place value iteration on a 5×5 gridworld, showing identical optimal values and policies, with comparable convergence performance.  
- **Problem 4** applied off-policy Monte Carlo with weighted importance sampling, estimating values and policies without using the model and comparing them with value iteration results.  

Together, these problems highlight the trade-off between **model-based methods** (exact, fast when transitions are known) and **model-free methods** (flexible, data-driven, but slower and noisier). This demonstrates the strengths and limitations of different reinforcement learning approaches in practice.
